# Web Scraping - Job Posting

In [1]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [41]:
url = "https://www.accenture.com/in-en/careers/jobdetails?id=AIOC-S01577479_en&title=Analytics+and+Modeling+Analyst"
loader = WebBaseLoader(url)
page_data = loader.load().pop().page_content

In [42]:
print(page_data)






Analytics and Modeling Analyst


























































































































Skip to main content
Skip to footer













Menu




 
Accenture


 


Accenture








Close Menu





What we do

 





Back



What we do





Capabilities
Capabilities



Cloud 
					


Customer Service 
					


Cybersecurity 
					


Data and Artificial Intelligence 
					


Digital Engineering and Manufacturing 
					


Ecosystem Partners 
					


Emerging Technology 
					


Finance and Risk Management 
					


Infrastructure and Capital Projects 
					


Learning 
					


Managed Services 
					


Marketing and Experience 
					


Metaverse 
					


Sales and Commerce 
					


Strategy 
					


Supply Chain 
					


Sustainability 
					


Talent and Organization 
					


Technology Transformation 
					





Industries
Industries



Aerospace and Defense 
					


Automotive 
					


Banking 
					


Capital Markets

## Loading llm from groq

In [9]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(

    temperature = 0,
    groq_api_key = "<your_key>",
    model_name = "llama-3.1-8b-instant"

)

## Formatting scraped data

In [44]:
from langchain_core.prompts import PromptTemplate

In [45]:
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: ["role", "experience", "skills" and "description".]
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

In [46]:
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input = {'page_data':page_data})

In [47]:
print(res.content)

```json
[
  {
    "role": "Analytics and Modeling Analyst",
    "experience": "2-5 years",
    "skills": "Insight Engine - Analytics Insights",
    "description": "Job Description\n\nSkill required: Insight Engine - Analytics Insights\nDesignation: Analytics and Modeling Analyst\nQualifications:Any Graduation  Years of Experience:3 to 5 years  About AccentureAccenture is a global professional services company with leading capabilities in digital, cloud and security.Combining unmatched experience and specialized skills across more than 40 industries, we offer Strategy and Consulting, Technology and Operations services, and Accenture Song— all powered by the world’s largest network of Advanced Technology and  Intelligent Operations centers. Our 699,000 people deliver on the promise of technology and human ingenuity every day, serving clients in more than 120 countries.  We embrace the power of change to create value and shared success for our clients, people, shareholders, partners and  

In [50]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res


[{'role': 'Analytics and Modeling Analyst',
  'experience': '2-5 years',
  'skills': 'Insight Engine - Analytics Insights',
  'description': 'Job Description\n\nSkill required: Insight Engine - Analytics Insights\nDesignation: Analytics and Modeling Analyst\nQualifications:Any Graduation  Years of Experience:3 to 5 years  About AccentureAccenture is a global professional services company with leading capabilities in digital, cloud and security.Combining unmatched experience and specialized skills across more than 40 industries, we offer Strategy and Consulting, Technology and Operations services, and Accenture Song— all powered by the world’s largest network of Advanced Technology and  Intelligent Operations centers. Our 699,000 people deliver on the promise of technology and human ingenuity every day, serving clients in more than 120 countries.  We embrace the power of change to create value and shared success for our clients, people, shareholders, partners and  communities.Visit us a

In [54]:
job = json_res[0]

In [56]:
job['skills']

'Insight Engine - Analytics Insights'

# Adding data to chromaDB

In [33]:
import pandas as pd 

In [34]:
df = pd.read_csv("my_portfolio.csv")
df.shape

(20, 2)

In [35]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])


In [57]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links


[[{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

# Generate email 

In [61]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Kiran, a business development executive at ABCDEF. ABCDEF is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of ABCDEF 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase ABCDEF's portfolio: {link_list}
        Remember you are Kiran, BDE at ABCDEF. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )


In [62]:
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})

In [63]:
print(res.content)

Subject: Unlock the Power of AI and Machine Learning with ABCDEF

Dear Hiring Manager,

I came across the job description for a Machine Learning Data Scientist at Accenture, and I was impressed by the requirements and responsibilities outlined. As a Business Development Executive at ABCDEF, an AI & Software Consulting company, I believe our expertise and capabilities align perfectly with your needs.

At ABCDEF, we specialize in facilitating the seamless integration of business processes through automated tools, empowering numerous enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has a proven track record of delivering cutting-edge AI and machine learning solutions that drive business growth and innovation.

I'd like to highlight a few examples from our portfolio that demonstrate our capabilities in machine learning and data science:

* Our team has successfully developed and impleme